In [1]:
%matplotlib inline
import pysal as ps
import pandas as pd
import numpy as np

from shapely.geometry import Polygon, point
from geopandas import GeoSeries, GeoDataFrame, read_file
from pandas import DataFrame
import matplotlib.pyplot as plt
import geopandas as gpd

In [2]:
def calculateHitRate(values):
    n=values[3]
    N=values[5]
    HitRate = float(n)/float(N)
    return HitRate

def calculatePAI(values):
    n=values[3]
    N=values[5]
    a=values[0]
    A=values[1]
    PAI = ((float(n)/float(N))*100)/((float(a)/float(A)*100))
    return PAI

def calculateRRI(values):
    n1=values[2]
    n2=values[3]
    N1=values[4]
    N2=values[5]
    RRI = (float(n1)/float(n2))/(float(N1)/float(N2))
    return RRI


def prepareParameters(areas, hotspots, newpoints, oldpoints):
    values = []    
    a = A = n1 = n2 = N1 = N2 = 0  
    a = a + sum(hotspots['geometry'].area)/1000000
    A = A + sum(areas['geometry'].area)/1000000    
    N1=len(oldpoints)
    N2=len(newpoints)    
    count = 0
    for i, h in hotspots.iterrows():
        for j, p in newpoints.iterrows():
            if h.geometry.contains(p.geometry):
                count = count + 1
                newpoints = newpoints.drop([j])
    n2=count
    count = 0
    for i, h in hotspots.iterrows():
        for j, p in oldpoints.iterrows():
            if h.geometry.contains(p.geometry):
                count = count + 1
                oldpoints = oldpoints.drop([j])
    n1=count
    values.append(a)
    values.append(A)
    values.append(n1)#previous points in previous hotspots
    values.append(n2)#new points in previous hotspots
    values.append(N1)#all previous points
    values.append(N2)#all new points     
    return values

In [3]:
#load hotspots
#load stac
STACDamageAndArson1 = read_file('CrimeStatOutput/ST1_2017DamageAndArson1STAC.shp')
STACBurglary1 = read_file('CrimeStatOutput/ST1_2017Burglary1STAC.shp')
STACPublicOrder1 = read_file('CrimeStatOutput/ST1_2017PublicOrder1STAC.shp')
STACVehicleCrime1 = read_file('CrimeStatOutput/ST1_2017VehicleCrime1STAC.shp')
#load nnh
NNHDamageAndArson1 = read_file('CrimeStatOutput/NNH1_2017DamageAndArson1NNH.shp')
NNHBurglary1 = read_file('CrimeStatOutput/NNH1_2017Burglary1NNH.shp')
NNHPublicOrder1 = read_file('CrimeStatOutput/NNH1_2017PublicOrder1NNH.shp')
NNHVehicleCrime1 = read_file('CrimeStatOutput/NNH1_2017VehicleCrime1NNH.shp')

#load kde
kdeDamageAndArson1 = read_file('DA117_LsoaZMax.shp')
kdeBurglary1 = read_file('Bur117_LsoaZMax.shp')
kdePublicOrder1 = read_file('Pub117_LsoaZMax.shp')
kdeVehicleCrime1 = read_file('Veh117_LsoaZMax.shp')

#load local moran
LMoranDamageAndArson1 = read_file('DamageAndArson1_IMax.shp')
LMoranBurglary1 = read_file('Burglary1_IMax.shp')
LMoranPublicOrder1 = read_file('PublicOrder1_IMax.shp')
LMoranVehicleCrime1 = read_file('VehicleCrime1_IMax.shp')

#load local g
LG_DamageAndArson1 = read_file('DamageAndArson1_GMax.shp')
LG_Burglary1 = read_file('Burglary1_GMax.shp')
LG_PublicOrder1 = read_file('PublicOrder1_GMax.shp')
LG_VehicleCrime1 = read_file('VehicleCrime1_GMax.shp')

areas = read_file('data/Clipped.shp')

In [3]:
#load hotspots
#load stac
STACAntisocial117 = read_file('CrimeStatOutput/ST1_2017Antisocia1STAC.shp')
STACViolenceAndSex117 = read_file('CrimeStatOutput/ST1_2017ViolenceAndSex1STAC.shp')

#load nnh
NNHAntisocial117 = read_file('CrimeStatOutput/NNH1_2017Antisocia1NNH.shp')
NNHViolenceAndSex117 = read_file('CrimeStatOutput/NNH1_2017ViolenceAndSex1NNH.shp')

#load kde
kdeAntisocial117 = read_file('AS117_LsoaZMax.shp')
kdeViolenceAndSex117 = read_file('VS117_LsoaZMax.shp')

#load local moran
LMoranAntisocial117 = read_file('Antisocial117_LsoaIMax.shp')
LMoranViolenceAndSex117 = read_file('ViolenceAndSex117_LsoaIMax.shp')

#load local g
LG_Antisocial117 = read_file('Antisocial117_LSOAGMax.shp')
LG_ViolenceAndSex117 = read_file('ViolenceAndSex117_LSOAGMax.shp')

areas = read_file('data/Clipped.shp')

In [4]:
#this will prepare the incident data set for the new period
DamageAndArson2=read_file('2_2017DamageAndArson2.shp')
Burglary2=read_file('2_2017Burglary2.shp')
PublicOrder2=read_file('2_2017PublicOrder2.shp')
VehicleCrime2=read_file('2_2017VehicleCrime2.shp')

#this will prepare the incident data set for the old period
DamageAndArson1=read_file('1_2017DamageAndArson1.shp')
Burglary1=read_file('1_2017Burglary1.shp')
PublicOrder1=read_file('1_2017PublicOrder1.shp')
VehicleCrime1=read_file('1_2017VehicleCrime1.shp')

In [4]:
#this will prepare the incident data set for the new period
Antisocial2=read_file('1_2017Antisocia2.shp')
ViolenceAndSex2=read_file('1_2017ViolenceAndSex2.shp')

#this will prepare the incident data set for the old period
Antisocial1=read_file('1_2017Antisocia1.shp')
ViolenceAndSex1=read_file('1_2017ViolenceAndSex1.shp')

In [5]:
STAC_DA_VAlues = prepareParameters(areas, STACDamageAndArson1, DamageAndArson2, DamageAndArson1)
STAC_Bur_Values = prepareParameters(areas, STACBurglary1, Burglary2, Burglary1)
STAC_Pub_VAlues = prepareParameters(areas, STACPublicOrder1, PublicOrder2, PublicOrder1)
STAC_Veh_Values = prepareParameters(areas, STACVehicleCrime1, VehicleCrime2, VehicleCrime1)
print "finihsed with STAC"

NNH_DA_VAlues = prepareParameters(areas, NNHDamageAndArson1, DamageAndArson2, DamageAndArson1)
NNH_Bur_Values = prepareParameters(areas, NNHBurglary1, Burglary2, Burglary1)
NNH_Pub_VAlues = prepareParameters(areas, NNHPublicOrder1, PublicOrder2, PublicOrder1)
NNH_Veh_Values = prepareParameters(areas, NNHVehicleCrime1, VehicleCrime2, VehicleCrime1)
print "finihsed with NNH"

KDE_DA_VAlues = prepareParameters(areas, kdeDamageAndArson1, DamageAndArson2, DamageAndArson1)
KDE_Bur_Values = prepareParameters(areas, kdeBurglary1, Burglary2, Burglary1)
KDE_Pub_VAlues = prepareParameters(areas, kdePublicOrder1, PublicOrder2, PublicOrder1)
KDE_Veh_Values = prepareParameters(areas, kdeVehicleCrime1, VehicleCrime2, VehicleCrime1)
print "finihsed with KDE"

Moran_DA_VAlues = prepareParameters(areas, LMoranDamageAndArson1, DamageAndArson2, DamageAndArson1)
Moran_Bur_Values = prepareParameters(areas, LMoranBurglary1, Burglary2, Burglary1)
Moran_Pub_VAlues = prepareParameters(areas, LMoranPublicOrder1, PublicOrder2, PublicOrder1)
Moran_Veh_Values = prepareParameters(areas, LMoranVehicleCrime1, VehicleCrime2, VehicleCrime1)
print "finihsed with Local Moran"

G_DA_VAlues = prepareParameters(areas, LG_DamageAndArson1, DamageAndArson2, DamageAndArson1)
G_Bur_Values = prepareParameters(areas, LG_Burglary1, Burglary2, Burglary1)
G_Pub_VAlues = prepareParameters(areas, LG_PublicOrder1, PublicOrder2, PublicOrder1)
G_Veh_Values = prepareParameters(areas, LG_VehicleCrime1, VehicleCrime2, VehicleCrime1)
print "finihsed with Local G"

finihsed with STAC
finihsed with NNH
finihsed with KDE
finihsed with Local Moran
finihsed with Local G


In [13]:
HitRateDA_Values=[]
HitRateBur_Values=[]
HitRatePub_Values=[]
HitRateVeh_Values=[]

HitRateDA_Values.append(calculateHitRate(STAC_DA_VAlues))
HitRateDA_Values.append(calculateHitRate(NNH_DA_VAlues))
HitRateDA_Values.append(calculateHitRate(KDE_DA_VAlues))
HitRateDA_Values.append(calculateHitRate(Moran_DA_VAlues))
HitRateDA_Values.append(calculateHitRate(G_DA_VAlues))

HitRateBur_Values.append(calculateHitRate(STAC_Bur_Values))
HitRateBur_Values.append(calculateHitRate(NNH_Bur_Values))
HitRateBur_Values.append(calculateHitRate(KDE_Bur_Values))
HitRateBur_Values.append(calculateHitRate(Moran_Bur_Values))
HitRateBur_Values.append(calculateHitRate(G_Bur_Values))

HitRatePub_Values.append(calculateHitRate(STAC_Pub_VAlues))
HitRatePub_Values.append(calculateHitRate(NNH_Pub_VAlues))
HitRatePub_Values.append(calculateHitRate(KDE_Pub_VAlues))
HitRatePub_Values.append(calculateHitRate(Moran_Pub_VAlues))
HitRatePub_Values.append(calculateHitRate(G_Pub_VAlues))

HitRateVeh_Values.append(calculateHitRate(STAC_Veh_Values))
HitRateVeh_Values.append(calculateHitRate(NNH_Veh_Values))
HitRateVeh_Values.append(calculateHitRate(KDE_Veh_Values))
HitRateVeh_Values.append(calculateHitRate(Moran_Veh_Values))
HitRateVeh_Values.append(calculateHitRate(G_Veh_Values))


PAIDA_Values=[]
PAIBur_Values=[]
PAIPub_Values=[]
PAIVeh_Values=[]
PAIDA_Values.append(calculatePAI(STAC_DA_VAlues))
PAIDA_Values.append(calculatePAI(NNH_DA_VAlues))
PAIDA_Values.append(calculatePAI(KDE_DA_VAlues))
PAIDA_Values.append(calculatePAI(Moran_DA_VAlues))
PAIDA_Values.append(calculatePAI(G_DA_VAlues))

PAIBur_Values.append(calculatePAI(STAC_Bur_Values))
PAIBur_Values.append(calculatePAI(NNH_Bur_Values))
PAIBur_Values.append(calculatePAI(KDE_Bur_Values))
PAIBur_Values.append(calculatePAI(Moran_Bur_Values))
PAIBur_Values.append(calculatePAI(G_Bur_Values))

PAIPub_Values.append(calculatePAI(STAC_Pub_VAlues))
PAIPub_Values.append(calculatePAI(NNH_Pub_VAlues))
PAIPub_Values.append(calculatePAI(KDE_Pub_VAlues))
PAIPub_Values.append(calculatePAI(Moran_Pub_VAlues))
PAIPub_Values.append(calculatePAI(G_Pub_VAlues))

PAIVeh_Values.append(calculatePAI(STAC_Veh_Values))
PAIVeh_Values.append(calculatePAI(NNH_Veh_Values))
PAIVeh_Values.append(calculatePAI(KDE_Veh_Values))
PAIVeh_Values.append(calculatePAI(Moran_Veh_Values))
PAIVeh_Values.append(calculatePAI(G_Veh_Values))


RRIDA_Values=[]
RRIBur_Values=[]
RRIPub_Values=[]
RRIVeh_Values=[]
RRIDA_Values.append(calculateRRI(STAC_DA_VAlues))
RRIDA_Values.append(calculateRRI(NNH_DA_VAlues))
RRIDA_Values.append(calculateRRI(KDE_DA_VAlues))
RRIDA_Values.append(calculateRRI(Moran_DA_VAlues))
RRIDA_Values.append(calculateRRI(G_DA_VAlues))

RRIBur_Values.append(calculateRRI(STAC_Bur_Values))
RRIBur_Values.append(calculateRRI(NNH_Bur_Values))
RRIBur_Values.append(calculateRRI(KDE_Bur_Values))
RRIBur_Values.append(calculateRRI(Moran_Bur_Values))
RRIBur_Values.append(calculateRRI(G_Bur_Values))

RRIPub_Values.append(calculateRRI(STAC_Pub_VAlues))
RRIPub_Values.append(calculateRRI(NNH_Pub_VAlues))
RRIPub_Values.append(calculateRRI(KDE_Pub_VAlues))
RRIPub_Values.append(calculateRRI(Moran_Pub_VAlues))
RRIPub_Values.append(calculateRRI(G_Pub_VAlues))

RRIVeh_Values.append(calculateRRI(STAC_Veh_Values))
RRIVeh_Values.append(calculateRRI(NNH_Veh_Values))
RRIVeh_Values.append(calculateRRI(KDE_Veh_Values))
RRIVeh_Values.append(calculateRRI(Moran_Veh_Values))
RRIVeh_Values.append(calculateRRI(G_Veh_Values))


#Build table for HitRate relusts
HitRateValues = pd.DataFrame()
HitRateValues['Method']=GeoSeries(['STAC', 'NNHC','KDE','Local Moran', 'Local G'])
HitRateValues['DamageAndArson']=GeoSeries(HitRateDA_Values)
HitRateValues['Burglary']=GeoSeries(HitRateBur_Values)
HitRateValues['PublicOrder']=GeoSeries(HitRatePub_Values)
HitRateValues['VehicleCrime']=GeoSeries(HitRateVeh_Values)

#Build table for PAI results
PAIValues = pd.DataFrame()
PAIValues['Method']=GeoSeries(['STAC', 'NNHC','KDE','Local Moran', 'Local G'])
PAIValues['DamageAndArson']=GeoSeries(PAIDA_Values)
PAIValues['Burglary']=GeoSeries(PAIBur_Values)
PAIValues['PublicOrder']=GeoSeries(PAIPub_Values)
PAIValues['VehicleCrime']=GeoSeries(PAIVeh_Values)

#Build table for RRI results
RRIValues = pd.DataFrame()
RRIValues['Method']=GeoSeries(['STAC', 'NNHC','KDE','Local Moran', 'Local G'])
RRIValues['DamageAndArson']=GeoSeries(RRIDA_Values)
RRIValues['Burglary']=GeoSeries(RRIBur_Values)
RRIValues['PublicOrder']=GeoSeries(RRIPub_Values)
RRIValues['VehicleCrime']=GeoSeries(RRIVeh_Values)

HitRateValues.to_csv('AccuracyMeasures/HitRateRestOfCrimes.csv', index=False)
PAIValues.to_csv('AccuracyMeasures/PAIValuesRestOfCrimes.csv', index=False)
RRIValues.to_csv('AccuracyMeasures/RRIValuesRestOfCrimes.csv', index=False)

In [5]:
STAC_VS_VAlues = prepareParameters(areas, STACViolenceAndSex117, ViolenceAndSex2, ViolenceAndSex1)
STAC_AS_Values = prepareParameters(areas, STACAntisocial117, Antisocial2, Antisocial1)
print "finihsed with STAC"

NNH_VS_VAlues = prepareParameters(areas, NNHViolenceAndSex117, ViolenceAndSex2, ViolenceAndSex1)
NNH_AS_Values = prepareParameters(areas, NNHAntisocial117, Antisocial2, Antisocial1)
print "finihsed with NNH"

KDE_VS_VAlues = prepareParameters(areas, kdeViolenceAndSex117, ViolenceAndSex2, ViolenceAndSex1)
KDE_AS_Values = prepareParameters(areas, kdeAntisocial117, Antisocial2, Antisocial1)
print "finihsed with KDE"

Moran_VS_VAlues = prepareParameters(areas, LMoranViolenceAndSex117, ViolenceAndSex2, ViolenceAndSex1)
Moran_AS_Values = prepareParameters(areas, LMoranAntisocial117, Antisocial2, Antisocial1)
print "finihsed with Local Moran"

G_VS_VAlues = prepareParameters(areas, LG_ViolenceAndSex117, ViolenceAndSex2, ViolenceAndSex1)
G_AS_Values = prepareParameters(areas, LG_Antisocial117, Antisocial2, Antisocial1)
print "finihsed with Local G"


finihsed with STAC
finihsed with NNH
finihsed with KDE
finihsed with Local Moran
finihsed with Local G


In [6]:
HitRateAtinSocialValues=[]
HitRateVS_Values=[]
HitRateAtinSocialValues.append(calculateHitRate(STAC_AS_Values))
HitRateAtinSocialValues.append(calculateHitRate(NNH_AS_Values))
HitRateAtinSocialValues.append(calculateHitRate(KDE_AS_Values))
HitRateAtinSocialValues.append(calculateHitRate(Moran_AS_Values))
HitRateAtinSocialValues.append(calculateHitRate(G_AS_Values))
HitRateVS_Values.append(calculateHitRate(STAC_VS_VAlues))
HitRateVS_Values.append(calculateHitRate(NNH_VS_VAlues))
HitRateVS_Values.append(calculateHitRate(KDE_VS_VAlues))
HitRateVS_Values.append(calculateHitRate(Moran_VS_VAlues))
HitRateVS_Values.append(calculateHitRate(G_VS_VAlues))


PAIAtinSocialValues=[]
PAIVS_Values=[]
PAIAtinSocialValues.append(calculatePAI(STAC_AS_Values))
PAIAtinSocialValues.append(calculatePAI(NNH_AS_Values))
PAIAtinSocialValues.append(calculatePAI(KDE_AS_Values))
PAIAtinSocialValues.append(calculatePAI(Moran_AS_Values))
PAIAtinSocialValues.append(calculatePAI(G_AS_Values))
PAIVS_Values.append(calculatePAI(STAC_VS_VAlues))
PAIVS_Values.append(calculatePAI(NNH_VS_VAlues))
PAIVS_Values.append(calculatePAI(KDE_VS_VAlues))
PAIVS_Values.append(calculatePAI(Moran_VS_VAlues))
PAIVS_Values.append(calculatePAI(G_VS_VAlues))


RRIAtinSocialValues=[]
RRIVS_Values=[]
RRIAtinSocialValues.append(calculateRRI(STAC_AS_Values))
RRIAtinSocialValues.append(calculateRRI(NNH_AS_Values))
RRIAtinSocialValues.append(calculateRRI(KDE_AS_Values))
RRIAtinSocialValues.append(calculateRRI(Moran_AS_Values))
RRIAtinSocialValues.append(calculateRRI(G_AS_Values))
RRIVS_Values.append(calculateRRI(STAC_VS_VAlues))
RRIVS_Values.append(calculateRRI(NNH_VS_VAlues))
RRIVS_Values.append(calculateRRI(KDE_VS_VAlues))
RRIVS_Values.append(calculateRRI(Moran_VS_VAlues))
RRIVS_Values.append(calculateRRI(G_VS_VAlues))


#Build table for HitRate relusts
HitRateValues = pd.DataFrame()
HitRateValues['Method']=GeoSeries(['STAC', 'NNHC','KDE','Local Moran', 'Local G'])
HitRateValues['Anti-Social Crimes']=GeoSeries(HitRateAtinSocialValues)
HitRateValues['Violence And Sex Crimes']=GeoSeries(HitRateVS_Values)

#Build table for PAI results
PAIValues = pd.DataFrame()
PAIValues['Method']=GeoSeries(['STAC', 'NNHC','KDE','Local Moran', 'Local G'])
PAIValues['Anti-Social Crimes']=GeoSeries(PAIAtinSocialValues)
PAIValues['Violence And Sex Crimes']=GeoSeries(PAIVS_Values)

#Build table for RRI results
RRIValues = pd.DataFrame()
RRIValues['Method']=GeoSeries(['STAC', 'NNHC','KDE','Local Moran', 'Local G'])
RRIValues['Anti-Social Crimes']=GeoSeries(RRIAtinSocialValues)
RRIValues['Violence And Sex Crimes']=GeoSeries(RRIVS_Values)

In [7]:
HitRateValues.to_csv('AccuracyMeasures/HitRate.csv', index=False)
PAIValues.to_csv('AccuracyMeasures/PAIValues.csv', index=False)
RRIValues.to_csv('AccuracyMeasures/RRIValues.csv', index=False)

In [8]:
HitRateValues

,Method,Anti-Social Crimes,Violence And Sex Crimes
0,STAC,0.098501,0.073059
1,NNHC,0.152626,0.106201
2,KDE,0.412047,0.431580
3,Local Moran,0.468063,0.483429
4,Local G,0.368253,0.332597


In [9]:
PAIValues

,Method,Anti-Social Crimes,Violence And Sex Crimes
0,STAC,29.240192,20.203621
1,NNHC,9.260134,7.419564
2,KDE,1.225540,1.360857
3,Local Moran,2.007000,2.000365
4,Local G,4.289739,4.482209


In [11]:
RRIValues

,Method,Anti-Social Crimes,Violence And Sex Crimes
0,STAC,1.016044,1.039079
1,NNHC,1.145521,1.055330
2,KDE,1.020134,0.991109
3,Local Moran,1.102268,1.128641
4,Local G,1.095603,1.122148
